941. Valid Mountain Array

https://leetcode.cn/problems/valid-mountain-array/

## 一、题目解读与核心注意事项

这道题要求我们判断一个整数数组是否为“山脉数组”。一个有效的山脉数组必须同时满足三个条件：
1.  数组长度 `arr.length >= 3`。
2.  存在一个唯一的“山顶”，其索引 `i` 满足 `0 < i < arr.length - 1`。
3.  从数组开头到山顶是**严格递增**的。
4.  从山顶到数组末尾是**严格递减**的。

### ⚠️ **核心注意事项：为什么纯递增或纯递减数组返回 False？**

这是本题最关键的陷阱。原因在于对“山顶”索引 `i` 的严格限制：**`0 < i < arr.length - 1`**。

* **`i > 0`** 意味着山顶**不能是**数组的第一个元素。这直接排除了所有**纯递减**的数组。
* **`i < arr.length - 1`** 意味着山顶**不能是**数组的最后一个元素。这直接排除了所有**纯递增**的数组。

一个真正的“山脉”，必须有明确的“上坡路”和“下坡路”。纯递增数组只有上坡，纯递减数组只有下坡，因此它们都不是有效的山脉数组。

---

## 二、解法一：单次遍历法（模拟爬山）

该方法非常直观，完全模拟了“先上山，再下山”的完整过程。

### 思路分析

1.  **上坡**：用一个指针 `i` 从数组起点 `0` 开始向右遍历。只要数组是严格递增的 (`arr[i] < arr[i+1]`)，就继续前进。
2.  **检查山顶**：上坡结束后，指针 `i` 就停在了山顶。此时必须检查山顶的合法性：
    * 如果 `i` 还在起点 `0` (说明没有上坡路)。
    * 或者 `i` 已经到达终点 `n-1` (说明只有上坡路，没有下坡路)。
    * 以上两种情况均不构成山脉，返回 `False`。
3.  **下坡**：从山顶位置 `i` 继续向右遍历。只要数组是严格递减的 (`arr[i] > arr[i+1]`)，就继续前进。
4.  **验证终点**：如果下坡过程顺利走完，指针 `i` 的最终位置应该恰好是数组的最后一个索引 `n-1`。以此作为最终的判断依据。

### 代码实现

```python
from typing import List

class Solution:
    def validMountainArray_single_pass(self, arr: List[int]) -> bool:
        n = len(arr)
        if n < 3:
            return False

        i = 0

        # 1. 上坡：i 最终会停在山顶
        # 条件 i + 1 < n 确保了 arr[i + 1] 不会索引越界
        while i + 1 < n and arr[i] < arr[i + 1]:
            i += 1

        # 2. 检查山顶是否合法
        if i == 0 or i == n - 1:
            return False

        # 3. 下坡
        # 条件 i + 1 < n 同样确保 arr[i + 1] 不会索引越界
        while i + 1 < n and arr[i] > arr[i + 1]:
            i += 1

        # 4. 检查是否走到了数组的尽头
        return i == n - 1
```

---

## 三、解法二：双指针法（两端向中间汇合）

通过两个指针分别从数组的两端“爬坡”，看它们是否能在同一个山顶相遇。

### 思路分析

1.  **左指针上坡**：初始化左指针 `left = 0`。只要满足严格递增条件，就将 `left` 向右移动。
2.  **右指针上坡**：初始化右指针 `right = n - 1`。只要满足严格递增条件（从右往左看），就将 `right` 向左移动。
3.  **检查汇合点**：
    * 两个指针的移动都结束后，它们应该停在各自方向的山顶上。
    * 如果 `left` 和 `right` 相等，说明左右两边汇合于同一个山顶。
    * 并且，这个山顶不能是数组的端点（`left > 0` 且 `left < n - 1`），以确保同时存在上坡和下坡。

### 索引越界防护说明

在 `while` 循环的条件判断中：
* `left < n - 1` (或者 `left != n - 1`): 这个条件**必须放在前面**。它利用了 Python 的“短路求值”特性。如果 `left` 已经到达了最后一个索引，该条件为 `False`，循环会立即终止，**不会**再去执行后面的 `arr[left] < arr[left + 1]`，从而完美地避免了 `arr[n]` 的索引越界错误。
* `right > 0` : 同理，此条件也必须放在前面，以避免当 `right` 为 `0` 时，去访问 `arr[-1]` 导致错误。

### 代码实现

```python
from typing import List

class Solution:
    def validMountainArray(self, arr: List[int]) -> bool:
        n = len(arr)
        if n < 3:
            return False

        left, right = 0, n - 1

        # 左指针上坡，left < n - 1 用于防止 arr[left + 1] 越界
        while left < n - 1 and arr[left] < arr[left + 1]:
            left += 1
        
        # 右指针上坡（从右向左），right > 0 用于防止 arr[right - 1] 越界
        while right > 0 and arr[right] < arr[right - 1]:
            right -= 1

        # 检查山顶是否在数组内部，并且左右指针是否在同一点相遇
        # left > 0 保证有上坡
        # right < n - 1 保证有下坡
        # left == right 保证是单峰
        return left > 0 and right < n-1 and left == right
```

---

## 四、复杂度分析

两种方案的性能表现一致，都达到了最优。

* **时间复杂度**: $O(N)$
    * **单次遍历法**：指针 `i` 从头到尾最多遍历数组一次。
    * **双指针法**：`left` 指针和 `right` 指针合计最多也只遍历数组一次。
    * 因为必须检查数组中的每个元素才能确定其山脉结构，所以 $O(N)$ 是理论上的最佳时间复杂度。

* **空间复杂度**: $O(1)$
    * 两种方法都只使用了有限的几个变量（如 `n`, `i`, `left`, `right`），没有使用额外的、随输入数组大小而改变的数据结构。